# load the packages and functions

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import xarray as xr
import h5py
from scipy.io import loadmat
import matplotlib as mpl
import time
import gsw
from matplotlib.colors import TwoSlopeNorm

# import existing python files
plt.rcParams['figure.figsize'] = (10,4)

# add rdmds reading functions to path
sys.path.append("/home/mmurakami/MITgcm/MITgcm_c68r/MITgcm-checkpoint68r/utils/python/MITgcmutils/MITgcmutils/") # go to parent dir
from mds import *

# add the other files
sys.path.append("/home/mmurakami/crios_backups/an_helper_functions")
from read_binary import *
from calc_UV_conv_1face import calc_UV_conv_1face
from calc_mskmean_T_mod import calc_mskmean_T_mod
from mk3D_mod import mk3D_mod
from aste_helper_funcs import *
from timing_functions import *           # ts2dte, get_fnames, etc.
from binning import *                    # bin_array, create_mesh

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%run /home/mmurakami/crios_backups/ASTE_270/prep_grid.py

(50,) (50, 1350, 270)
hf1 (1350, 270)
(1, 1350, 270)
LwetC2d 146614
LwetC 4833023



# load the grid

In [4]:
# define basin we want
path = "/home/mmurakami/crios_backups/ASTE_270/offline_binning/sample_images/"

In [5]:
iB = 6    # read from BarentsSea
    
# mymsk below defines as all Arctic down to Fram Strait and BSO but not GINs Seas
mymsk = mskBasin.copy()

# Create a boolean mask for elements that are 6 or less
mask = mymsk == 6

# Set elements that are greater than 6 to np.nan
mymsk[mask] = 1
mymsk[~mask] = np.nan

# Get the number of points where mskBasin is 6 or less
npoints = np.count_nonzero(mymsk)  # Count the number of True values in the mask
print(npoints)

364500


In [6]:
mymsk3 = np.tile(mymsk[np.newaxis,:,:],(nz,1,1)) * mygrid['hFacC']
mymsk3.shape

(50, 1350, 270)

# timesteps

In [7]:
# create an array of the time steps we want to read
# use ts2dte to get december 2014
# first make an array of filenames
dt_aste = 600
startyr = 2002
endyr = 2019

# all the filenames in the system
fnames = get_fnames(dt_aste,startyr,endyr)

# ocean and ice
AB_gT=0
AB_gS=0

In [8]:
# dates

In [9]:
# years = list(np.arange(2003,2018,1))  # 15 year period
years = [2003,2004]
years = [str(i) for i in years]
years = np.array(years)
# years

In [10]:
# write the datetimes for the later period
times = {}

for year in years:
    times[year] = np.arange(1,13,1)   # write all the months for this example 5-year period

tsstr,datetimes = get_tsteps(times,fnames,dt_aste,startyr,1,1)
tsstr.shape
datetimes.shape

(24,)

In [11]:
# for one year
# tsstr = tsstr[:7]
# datetimes = datetimes[:7]
tsstr = tsstr[6:13]
datetimes = datetimes[6:13]
datetimes.shape

(7,)

In [12]:
# 'datetimes' is a list of datetime objects
dt = [(datetimes[i+1] - datetimes[i]).total_seconds() for i in range(len(datetimes) - 1)]
dt = np.array(dt)
dt.shape

(6,)

# create the J terms

In general, when we create the time-mean for these terms we are interested in the mean weighted by depth

### For heat

In [13]:
# start with the tendency
# read thetadr
file_name = 'budg3d_snap_set2'
meta_budg3d_snap_set2 = parsemeta(dirIn + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_snap_set2["fldList"])
varnames = np.array(["THETADR"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])

THETADR = np.full((len(tsstr),nz,ny,nx),np.nan)
for i in range(len(tsstr)):
    thisTHETADR,its,meta = rdmds(os.path.join(dirIn, file_name),int(tsstr[i]),returnmeta=True,rec=recs[0])
    thisTHETADR = thisTHETADR.reshape(nz,ny,nx)
    THETADR[i] = thisTHETADR

THETADR =  (THETADR[1:, :, :,:] - THETADR[:-1, :,:, :]) / dt[:,np.newaxis,np.newaxis,np.newaxis]    # PSU.m/s    # degC.m/s
THETADR.shape

(6, 50, 1350, 270)

In [14]:
tmptend=myparms['rcp']*(THETADR-AB_gT)* np.tile(RAC[np.newaxis,np.newaxis,:,:],(len(tsstr)-1,nz,1,1))  # J/m^3.degC * degC.m/s * m^2 = J/s
budgO = {}
budgI = {}
budgO['heatfluxes'] = {}
budgI['heatfluxes'] = {}

budgO['heatfluxes']['tend'] = tmptend     # J/s

budgO['heattend'] = np.nansum(tmptend,axis=1)

In [15]:
del THETADR, tmptend

In [16]:
# read the files
file_name = 'budg2d_zflux_set1'
meta_budg2d_zflux_set1 = parsemeta(dirIn + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg2d_zflux_set1["fldList"])
varnames = np.array(["TFLUX","oceQsw","SItflux"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])

TFLUX = np.zeros((len(tsstr)-1,ny,nx))
oceQsw = np.zeros((len(tsstr)-1,ny,nx))
SItflux = np.zeros((len(tsstr)-1,ny,nx))
# loop and add to list
for t in range(len(tsstr[1:])):
    t2 = int(tsstr[t+1])  # +1 so we can read end of budg time steps
    tmpTFLUX,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[0])
    tmpoceQsw,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[1])
    tmpSItflux,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[2])
    TFLUX[t] = tmpTFLUX.reshape(ny,nx)
    oceQsw[t] = tmpoceQsw.reshape(ny,nx)
    SItflux[t] = tmpSItflux.reshape(ny,nx)

# note: the following works provided that the first 3 terms are definitely there
file_name = "budg2d_zflux_set2"
meta_budg2d_zflux_set2 = parsemeta(dirIn + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg2d_zflux_set2["fldList"])
varnames = np.array(["oceQnet","WTHMASS","SIaaflux","TRELAX"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
# read the files in a loop to get all timesteps
oceQnet = np.zeros((len(tsstr)-1,ny,nx))
WTHMASS = np.zeros((len(tsstr)-1,ny,nx))
SIaaflux = np.zeros((len(tsstr)-1,ny,nx))
TRELAX = np.zeros((len(tsstr)-1,ny,nx))
for t in range(len(tsstr[1:])):
    t2 = int(tsstr[t+1])
    tmpoceQnet,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[0])
    tmpWTHMASS,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[1])
    tmpSIaaflux,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[2])
    tmpTRELAX,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[3])
    oceQnet[t] = tmpoceQnet.reshape(ny,nx)
    WTHMASS[t] = tmpWTHMASS.reshape(ny,nx)
    SIaaflux[t] = tmpSIaaflux.reshape(ny,nx)
    TRELAX[t] = tmpTRELAX.reshape(ny,nx)

# note: will not work if these are defined, fix for future steps
varnames = np.array(["TRELAX","SIabflux","SIacflux","SIeprflx","SIfldflx"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    if len(irec[0]) > 0:
        recs = np.append(recs, irec[0][0])
# if len(recs) == 0:
SIabflux = np.zeros((len(tsstr)-1,ny, nx))
SIacflux = np.zeros((len(tsstr)-1,ny, nx))
SIeprflx = np.zeros((len(tsstr)-1,ny, nx))
SIfldflx = np.zeros((len(tsstr)-1,ny, nx))

In [17]:
if myparms['useNLFS'] == 0:
    print('do nothing, already read above')
else:
    WTHMASS=0*WTHMASS

In [18]:
geothFlux = 0

if myparms['SaltPlumeHeatFlux']:
    print(1)
else:
    SPforcT1=0*np.ones((len(tsstr)-1,ny,nx))
    oceEPtnd=0*np.ones((len(tsstr)-1,nz,ny,nx))

In [19]:
# read kpp tend and from 3d zflux
file_name = "budg3d_kpptend_set1"
meta_budg3d_kpptend_set1 = parsemeta(dirIn + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_kpptend_set1["fldList"])
varnames = np.array(["KPPg_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
# loop through and add to the list
KPPg_TH = np.zeros((len(tsstr)-1,nz,ny,nx))
for t in range(len(tsstr[1:])):
    t2 = int(tsstr[t+1])
    tmpKPPg_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[0])
    KPPg_TH[t] = tmpKPPg_TH.reshape(nz,ny,nx)

# now 3d zfluxes
file_name = "budg3d_zflux_set2"
meta_budg3d_zflux_set2 = parsemeta(dirIn + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_zflux_set2["fldList"])
varnames = np.array(["ADVr_TH","DFrE_TH","DFrI_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
# loop through the time steps and add
ADVr_TH = np.zeros((len(tsstr)-1,nz,ny,nx))
DFrE_TH = np.zeros((len(tsstr)-1,nz,ny,nx))
DFrI_TH = np.zeros((len(tsstr)-1,nz,ny,nx))
for t in range(len(tsstr[1:])):
    t2 = int(tsstr[t+1])

    tmpADVr_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[0])
    tmpDFrE_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[1])
    tmpDFrI_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[2])
    ADVr_TH[t] = tmpADVr_TH.reshape(nz,ny,nx)
    DFrE_TH[t] = tmpDFrE_TH.reshape(nz,ny,nx)
    DFrI_TH[t] = tmpDFrI_TH.reshape(nz,ny,nx)


KeyboardInterrupt



In [ ]:
budgO['heatzconv']=TFLUX+geothFlux+SPforcT1                           # W/m^2 = J/m^2/s
zconv_top_heat = TFLUX  * RAC     # W/m^2 * m^2 = J/s
budgI['heatzconv']=-(SItflux+TFLUX-TRELAX+SPforcT1)

if myparms['useNLFS']==0:
    budgO['heatzconv']=budgO['heatzconv']-myparms['rcp']*WTHMASS[:,:,:]     # degC.m/s * J/m^3degC = J/m^2.s

budgI['heatzconv']=budgI['heatzconv']-SIabflux+SIacflux+SIeprflx
if(myparms['SEAICEheatConsFix']==0):
    budgI['heatzconv']=budgI['heatzconv']+SIaaflux

In [ ]:
del TFLUX,SPforcT1,WTHMASS,SIabflux,SIacflux,SIeprflx

In [ ]:
nr = mygrid['RC'].shape[0]
trWtopADV = -(ADVr_TH) * myparms['rcp']         # J/s
trWtopDF = -(DFrE_TH+DFrI_TH) * myparms['rcp']  # J/s
trWtopKPP = -(KPPg_TH) * myparms['rcp']         # J/s
trWtop = trWtopADV + trWtopDF + trWtopKPP       # J/s
dd = mygrid['RF'][:-1]
swfrac = 0.62*np.exp(dd/0.6)+(1-0.62)*np.exp(dd/20)
swfrac[dd < -200] = 0

In [ ]:
a = (np.tile(RAC[np.newaxis,:,:],(len(tsstr)-1,1,1)) * oceQsw)
b = np.tile(swfrac[np.newaxis,:,np.newaxis,np.newaxis],(len(tsstr)-1,1,ny,nx))

In [ ]:
swtop = b * np.tile(a[:,np.newaxis,:,:],(1,nz,1,1))   # J/s

In [ ]:
# print(swtop.shape,trWtop.shape)

mskC=np.tile(mygrid['mskC'][np.newaxis,:,:,:],(len(tsstr)-1,1,1,1))
print(mskC.shape)
swtop[np.isnan(mskC)]=0
trWtop=trWtop+swtop  # 323

In [ ]:
trWtop[:,0,:,:]=budgO['heatzconv']*np.tile(RAC[np.newaxis,:,:],(len(tsstr)-1,1,1))
trWbot = np.zeros_like(trWtop)
trWbot[:,:-1,:,:]=trWtop[:,1:,:,:]

budgO["heatfluxes"]["trWtop"] = trWtop
budgO["heatfluxes"]["trWbot"] = trWbot

In [ ]:
del trWtop,trWbot

In [ ]:
budgI["heatfluxes"]["trWtop"] = -np.tile(RAC[np.newaxis,:,:],(len(tsstr)-1,1,1)) * (budgI["heatzconv"] + budgO["heatzconv"])
budgI["heatfluxes"]["trWbot"] = -np.tile(RAC[np.newaxis,:,:],(len(tsstr)-1,1,1)) * budgO["heatzconv"]
budgO['heatfluxes']['zconv']=budgO['heatfluxes']['trWtop']-budgO['heatfluxes']['trWbot']

budgO['heatzconv'] = np.tile(RAC[np.newaxis,:,:],(len(tsstr)-1,1,1))*budgO['heatzconv']  # J/s
budgI['heatzconv']=np.tile(RAC[np.newaxis,:,:],(len(tsstr)-1,1,1))*budgI['heatzconv']    # J/s
# budgOI['heatzconv']=budgO['heatzconv']+budgI['heatzconv']

In [ ]:
# read adv and dfe
file_name = "budg3d_hflux_set2"
meta_budg3d_hflux_set2 = parsemeta(dirIn + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_hflux_set2["fldList"])
varnames = np.array(["ADVx_TH","ADVy_TH","DFxE_TH","DFyE_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])

# do this onle for datetimes[1:] so we can set it in the file
heatfluxeshconv = np.zeros((len(tsstr)-1,nz,ny,nx))
advfluxeshconv = np.zeros((len(tsstr)-1,nz,ny,nx))
dffluxeshconv = np.zeros((len(tsstr)-1,nz,ny,nx))
                     
for t in range(len(tsstr[1:])):
    print(t)
    t2 = int(tsstr[t + 1])
    # read the files
    ADVx_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[0])
    ADVy_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[1])
    DFxE_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[2])
    DFyE_TH,its,meta = rdmds(os.path.join(dirIn, file_name),t2,returnmeta=True,rec=recs[3])

    tmpUo = myparms['rcp'] * (ADVx_TH + DFxE_TH)
    tmpVo = myparms['rcp'] * (ADVy_TH + DFyE_TH)
    tmpUo = tmpUo.reshape(nz,ny,nx)
    tmpVo = tmpVo.reshape(nz,ny,nx)
    
    # reshape and get the faces
    tmpUo = get_aste_faces(tmpUo,nfx,nfy)              
    tmpVo = get_aste_faces(tmpVo,nfx,nfy)

    # set in the larger array
    heatfluxeshconv[t] = calc_UV_conv_mod(nfx,nfy,tmpUo,tmpVo)

    # also do for adv and df
    tmpUo = get_aste_faces(ADVx_TH.reshape(nz,ny,nx),nfx,nfy)
    tmpVo = get_aste_faces(ADVy_TH.reshape(nz,ny,nx),nfx,nfy)
    advfluxeshconv[t] = calc_UV_conv_mod(nfx,nfy,tmpUo,tmpVo) * myparms['rcp']          #J/s

    tmpUo = get_aste_faces(DFxE_TH.reshape(nz,ny,nx),nfx,nfy)
    tmpVo = get_aste_faces(DFyE_TH.reshape(nz,ny,nx),nfx,nfy)
    dffluxeshconv[t] = calc_UV_conv_mod(nfx,nfy,tmpUo,tmpVo) * myparms['rcp']      #J/s

budgO['heatfluxes']['hconv'] = heatfluxeshconv
budgO['heatfluxes']['ADV_hconv'] = advfluxeshconv
budgO['heatfluxes']['DF_hconv'] = dffluxeshconv

In [ ]:
del heatfluxeshconv,advfluxeshconv,dffluxeshconv

In [ ]:
# do vertical convergence for ADV and DF terms
tmpadv = np.full((len(tsstr)-1,nz,ny,nx),np.nan)
tmpadv[:,:-1,:,:] = (trWtopADV[:,:-1] - trWtopADV[:,1:])              # for surface thru seafloor

Tconv = budgO['heatfluxes']['ADV_hconv'] + tmpadv
budgO['heatfluxes']['ADV_Tconv'] = Tconv                            # J/s, this is the advective arrow of T for a cell

In [ ]:
del Tconv,tmpadv

In [ ]:
# do vertical convergence for ADV and DF terms
tmpdf = np.full((len(tsstr)-1,nz,ny,nx),np.nan)
tmpdf[:,:-1,:,:] = (trWtopDF[:,:-1] - trWtopDF[:,1:])              # for surface thru seafloor

dfTconv = budgO['heatfluxes']['DF_hconv'] + tmpdf
budgO['heatfluxes']['DF_Tconv'] = dfTconv      # J/s, this is the diffusive arrow of T for a cell

In [ ]:
del dfTconv,tmpdf

In [ ]:
tmpkpp = np.full((len(tsstr)-1,nz,ny,nx),np.nan)
tmpkpp[:,:-1,:,:] = trWtopKPP[:,:-1] - trWtopKPP[:,1:]
budgO['heatfluxes']['KPP_Tconv'] = tmpkpp        # no horizontal component for this

In [ ]:
del tmpkpp

# create an example time series OF SALT for a given set of points

In [ ]:
# create the bins of TS data
# try new T bins where different sizes
refined_section = np.linspace(-3,8,93)
coarse_section = np.linspace(8,15,21,endpoint=False)
binsTH_edges = np.concatenate((refined_section,coarse_section[1:]))
binsTH_centers = (binsTH_edges[:-1] + binsTH_edges[1:])/2
nT = binsTH_edges.shape[0]-1

# do bi-sectional form for S
coarse_section = np.linspace(0, 28, 30, endpoint=False)
refined_section = np.linspace(28, 40, 83)
binsSLT_edges = np.concatenate((coarse_section, refined_section))
binsSLT_centers = (binsSLT_edges[:-1] + binsSLT_edges[1:])/2
nS = binsSLT_edges.shape[0]-1

Tbin,Sbin = np.meshgrid(binsTH_edges,binsSLT_edges)
Tbincent,Sbincent = np.meshgrid(binsTH_centers,binsSLT_centers)

binwidthT = binsTH_edges[1:] - binsTH_edges[:-1]
binwidthS = binsSLT_edges[1:] - binsSLT_edges[:-1]
dT,dS = np.meshgrid(binwidthT,binwidthS)
dT = dT.reshape(112,112,1)
dS = dS.reshape(112,112,1)

In [ ]:
# # read theta and salt averages from the t2 timestep (average)
# file_name = "state_3d_set1"
# meta_budg3d_kpptend_set1 = parsemeta(dirState + file_name + "." + tsstr[0] + ".meta")
# fldlist = np.array(meta_budg3d_kpptend_set1["fldList"])
# varnames = np.array(["THETA","SALT"])
# recs = np.array([])
# for var in varnames:
#     irec = np.where(fldlist == var)
#     recs = np.append(recs, irec[0][0])


# THETA = np.zeros((len(tsstr)-1,nz,ny,nx))
# SALT = np.zeros((len(tsstr)-1,nz,ny,nx))
# binned_theta = np.zeros((len(tsstr)-1,nz,ny,nx))
# binned_salt = np.zeros((len(tsstr)-1,nz,ny,nx))

# # loop and add to list
# for t in range(len(tsstr[1:])):
#     t2 = int(tsstr[t+1])  # +1 so we can read end of budg time steps

#     tmpTHETA,its,meta = rdmds(os.path.join(dirState, file_name),t2,returnmeta=True,rec=recs[0])
#     tmpSALT,its,meta = rdmds(os.path.join(dirState, file_name),t2,returnmeta=True,rec=recs[1])
#     tmpTHETA = tmpTHETA.reshape(nz,ny,nx)
#     tmpSALT = tmpSALT.reshape(nz,ny,nx)

#     # set in larger array
#     THETA[t] = tmpTHETA
#     SALT[t] = tmpSALT

#     # bin for temp
#     tmpbinned_theta = bin_array(tmpTHETA,binsTH_edges)
#     tmpbinned_theta = tmpbinned_theta.astype(float)
#     tmpbinned_theta[tmpbinned_theta == nT] = np.nan
#     binned_theta[t] = tmpbinned_theta

#     # also for salt
#     tmpbinned_salt = bin_array(tmpSALT,binsSLT_edges)
#     tmpbinned_salt = tmpbinned_salt.astype(float)
#     tmpbinned_salt[tmpbinned_salt == nS] = np.nan
#     binned_salt[t] = tmpbinned_salt

In [ ]:
# del THETA,SALT

In [ ]:
# # also make arrays of the binned_theta and binned_salt widths
# default_value = 0
# binwidthT_map = np.full_like(binned_theta, default_value, dtype=np.float64)
# valid_mask = ~np.isnan(binned_theta)
# binwidthT_map[valid_mask] = binwidthT[binned_theta[valid_mask].astype(int)]

In [ ]:
# # for salt
# binwidthS_map = np.full_like(binned_theta, default_value, dtype=np.float64)
# binwidthS_map[valid_mask] = binwidthS[binned_salt[valid_mask].astype(int)]

# create points, the example points at the surface in the Barents Sea, so we can do a time series

In [ ]:
# get some example points for the basin

y_indices, x_indices = np.meshgrid(np.arange(ny), np.arange(nx), indexing='ij')
condition = (mskBasin == iB)

x_points = x_indices[condition]
y_points = y_indices[condition]

# # Combine the x and y points into a single array of shape (npoints, 2)
points = np.vstack((y_points, x_points)).T
points.shape

# do the same J terms breakdown for theta

In [ ]:
JtendTfull = np.zeros((len(tsstr)-1,nz,ny,nx))
JADVTfull = np.zeros((len(tsstr)-1,nz,ny,nx))
JDFTfull = np.zeros((len(tsstr)-1,nz,ny,nx))
JKPPTfull = np.zeros((len(tsstr)-1,nz,ny,nx))
JsurfTfull = np.zeros((len(tsstr)-1,nz,ny,nx))
budgetfull = np.zeros((len(tsstr)-1,nz,ny,nx))

for t in range(len(tsstr)-1):
    print(t)
    
    # show the vectors for each
    aT = budgO['heatfluxes']['tend'][t]  # J/s 
    bT = budgO['heatfluxes']['ADV_Tconv'][t]
    cT = budgO['heatfluxes']['DF_Tconv'][t]
    dT = budgO['heatfluxes']['KPP_Tconv'][t]
    eT = zconv_top_heat[t].reshape(1,ny,nx)
    fT = swtop[t]

    # translate this J terms for a column to the Jterms in 3D as a map

    # save the J terms for here for the single point   
    JtendT = (aT) / myparms['rcp']             # J/s / J/(m^3*degC) = degC.m^3/s
    JADVT = (bT) / myparms['rcp']
    JDFT = (cT) / myparms['rcp']
    JKPPT = (dT) / myparms['rcp']

    # do the surface term J vectors for the entire area
    JsurfT = np.zeros((nz,ny,nx))                    # initialize the surface term
    JsurfT[1:-1,:,:] += fT[2:,:,:] - fT[1:-1,:,:]    # add the convergence of the subsurface
    JsurfT[0,:,:] += fT[1]   # init the surface of the surface term without the TFLUX
    JsurfT[0,:,:] -= eT[0]
    JsurfT = JsurfT / myparms['rcp']

    # also do the budget based on the other terms
    budget_ofJ = JtendT - JADVT - JDFT - JKPPT + JsurfT
    #print(budget_ofJ.shape)

    JtendTfull[t] = JtendT
    JADVTfull[t] = JADVT
    JDFTfull[t] = JDFT
    JKPPTfull[t] = JKPPT
    JsurfTfull[t] = JsurfT
    budgetfull[t] = budget_ofJ
    

In [ ]:
del budgO

In [ ]:
# # write this to a dataset the first time
# ds = xr.Dataset(
#     {
#         "JtendTfull": (["time", "z", "y", "x"], JtendTfull)  # Replace 'variable_name' with the actual name
#     },
#     coords={
#         "time": datetimes[:-1],
#         "z": np.arange(nz),
#         "y": np.arange(ny),
#         "x": np.arange(nx),
#     }
# )

# # add the other terms
# ds['JADVTfull'] = (["time", "z", "y", "x"], JADVTfull)
# ds['JDFTfull'] = (["time", "z", "y", "x"], JDFTfull)
# ds['JKPPTfull'] = (["time", "z", "y", "x"], JKPPTfull)
# ds['JsurfTfull'] = (["time", "z", "y", "x"], JsurfTfull)
# ds['budgetfull'] = (["time", "z", "y", "x"], budgetfull)

# # write to a dataset (the first time, do not run again)
# ds.to_netcdf("earlyyears_J.nc", mode="w", format="NETCDF4", unlimited_dims=["time"], engine="netcdf4")

In [ ]:
# Open the existing NetCDF file to append if necessary
ds_old = xr.open_dataset("earlyyears_J.nc")

In [ ]:
datetimes

In [ ]:
# Convert the new data arrays into xarray DataArrays with appropriate dimensions
ds_new = xr.Dataset(
    {
        "JtendTfull": (["time", "z", "y", "x"], JtendTfull),  # Replace with new data arrays
        "JADVTfull": (["time", "z", "y", "x"], JADVTfull),
        "JDFTfull": (["time", "z", "y", "x"], JDFTfull),
        "JKPPTfull": (["time", "z", "y", "x"], JKPPTfull),
        "JsurfTfull": (["time", "z", "y", "x"], JsurfTfull),
        "budgetfull": (["time", "z", "y", "x"], budgetfull)
    },
    coords={
        "time": datetimes[:-1],  # New datetimes for the appended data
        "z": np.arange(nz),     # Assuming z, y, x remain the same
        "y": np.arange(ny),
        "x": np.arange(nx),
    }
)

# Append the new data to the existing NetCDF file

In [ ]:
ds_combined = xr.concat([ds_old,ds_new],dim="time")

In [ ]:
ds_combined

In [ ]:
# ds_combined.to_netcdf("earlyyears_J.nc", mode="w", format="NETCDF4", unlimited_dims=["time"], engine="netcdf4")

# time series stuff

In [ ]:
with open("heatterms.txt","a") as inf:
    
    for t in range(len(tsstr)-1):
        # get the values we want
        a = np.nansum((JADVTfull[t] * mymsk3))
        b = np.nansum((JDFTfull[t] * mymsk3))
        c = np.nansum((JKPPTfull[t] * mymsk3))
        d = np.nansum((JsurfTfull[t] * mymsk))
        e = np.nansum((JtendTfull[t] * mymsk3))

        # write to file
        inf.write(f"{datetimes[t]}\n")
        inf.write(f"{a}, {b}, {c}, {d}, {e}\n")
        inf.write(f"{a+b+c-d}\n")
inf.close()

In [ ]:
# read from this array for later

# Initialize lists to store the data
dates = []
abcde_values = []
abc_minus_d_values = []

# Open the file in read mode
with open("heatterms.txt", "r") as inf:
    while True:
        # Read the date/time line
        date_line = inf.readline().strip()
        if not date_line:  # End of file
            break
        dates.append(date_line[:7])
        
        # Read the a, b, c, d, e values line
        values_line = inf.readline().strip()
        a, b, c, d, e = map(float, values_line.split(", "))
        abcde_values.append([a, b, c, d, e])
        
        # Read the a+b+c-d value line
        result_line = inf.readline().strip()
        abc_minus_d_values.append(float(result_line))

# Convert the lists to NumPy arrays for easier manipulation
dates = np.array(dates)  # This will be an array of strings
abcde_values = np.array(abcde_values)  # This will be a (n, 5) array
abc_minus_d_values = np.array(abc_minus_d_values)  # This will be a (n,) array

In [ ]:
abcde_values.shape  # adv, df, kpp, surface, tend
abc_minus_d_values.shape

In [ ]:
dates = np.array(dates,dtype='datetime64[M]')

In [ ]:
# now we can look at the time series of the salt terms

# now we can look at a time series of each of the J terms for salt at this given cell in a given year

fig = plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.8)  # bigger hspace -- more vertical space

plt.suptitle(("Integrated Barents Sea Tendencies"))

ax = plt.subplot(611)
ax.plot(dates,abcde_values[:,-1])
coefficients = np.polyfit(np.arange(len(dates)), abcde_values[:,-1], 1)
trendline = np.polyval(coefficients, np.arange(len(dates)))
ax.plot(dates, trendline, label="Trendline", color='red', linestyle='--')
ax.set_title("heat tend")
ax.grid()
ax.set_ylabel("Sv/PSU")
num_ticks = len(dates)
tick_indices = np.arange(0, num_ticks, 12)  # Indices of every 12th tick
ax.set_xticks(dates[tick_indices])  # Set the positions of the ticks
ax.set_xticklabels(dates[tick_indices].astype(str), rotation=15, ha='right')  # Set the labels and format


ax = plt.subplot(612)
ax.plot(dates,abcde_values[:,0])
ax.set_title("ADV theta tend")
ax.grid()
ax.set_ylabel("Sv/PSU")
ax.set_xticks(dates[tick_indices])  # Set the positions of the ticks
ax.set_xticklabels(dates[tick_indices].astype(str), rotation=15, ha='right')  # Set the labels and format

ax = plt.subplot(613)
ax.plot(dates,abcde_values[:,1])
ax.set_title("DF theta tend")
ax.grid()
ax.set_ylabel("Sv/PSU")
ax.set_xticks(dates[tick_indices])  # Set the positions of the ticks
ax.set_xticklabels(dates[tick_indices].astype(str), rotation=15, ha='right')  # Set the labels and format

ax = plt.subplot(614)
ax.plot(dates,abcde_values[:,2])
ax.set_title("KPP theta tend")
ax.grid()
ax.set_ylabel("Sv/PSU")
ax.set_xticks(dates[tick_indices])  # Set the positions of the ticks
ax.set_xticklabels(dates[tick_indices].astype(str), rotation=15, ha='right')  # Set the labels and format

ax = plt.subplot(615)
ax.plot(dates,abcde_values[:,3])
ax.set_title("surf theta tend")
ax.grid()
ax.set_ylabel("Sv/PSU")
ax.set_xticks(dates[tick_indices])  # Set the positions of the ticks
ax.set_xticklabels(dates[tick_indices].astype(str), rotation=15, ha='right')  # Set the labels and format

ax = plt.subplot(616)
ax.plot(dates,abc_minus_d_values)
ax.set_title("tend from terms")
ax.grid()
ax.set_ylabel("Sv/PSU")
ax.set_xticks(dates[tick_indices])  # Set the positions of the ticks
ax.set_xticklabels(dates[tick_indices].astype(str), rotation=15, ha='right')  # Set the labels and format

plt.savefig(path + "BarentsSea_heatJterms_ts.png",dpi=300)

In [ ]:
dates